In [143]:
#from pymystem3 import Mystem

!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!sudo cp mystem /root/.local/bin/mystem

import gensim
from pymystem3 import Mystem

stemmer = Mystem()
lemmas = stemmer.lemmatize("Красивая мама красиво мыла раму")
print(lemmas)

--2021-10-20 00:42:23--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.245, 5.45.205.244, 5.45.205.241, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.245|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cachev2-spb02.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz?lid=122 [following]
--2021-10-20 00:42:24--  https://cachev2-spb02.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz?lid=122
Resolving cachev2-spb02.cdn.yandex.net (cachev2-spb02.cdn.yandex.net)... 37.140.137.4, 2a02:6b8:0:2221::304
Connecting to cachev2-spb02.cdn.yandex.net (cachev2-spb02.cdn.yandex.net)|37.140.137.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.12’

mystem-3.0-linux3.1 100%

In [3]:
from google.colab import drive
drive.mount('/content/drive')

DRIVE_PATH = "/content/drive/MyDrive/YandexCup/"


Mounted at /content/drive


In [4]:
!pip install -q transformers
!pip install -q natasha
!pip install -q pymorphy2
!pip install -q -U pymorphy2-dicts-ru
!pip install -q hnswlib
!pip install -q pymystem3
!pip install -q pymorphy2
!pip install -q -U pymorphy2-dicts-ru
!pip install https://github.com/kpu/kenlm/archive/master.zip
!pip install -q kenlm


     |████████████████████████████████| 2.9 MB 15.7 MB/s 
     |████████████████████████████████| 596 kB 42.9 MB/s 
     |████████████████████████████████| 3.3 MB 57.6 MB/s 
     |████████████████████████████████| 895 kB 52.5 MB/s 
     |████████████████████████████████| 56 kB 6.1 MB/s 
     |████████████████████████████████| 34.4 MB 36 kB/s 
     |████████████████████████████████| 55 kB 5.1 MB/s 
     |████████████████████████████████| 49 kB 8.5 MB/s 
     |████████████████████████████████| 41 kB 152 kB/s 
     |████████████████████████████████| 8.2 MB 45.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     | 540 kB 4.9 MB/s
  Created wheel for kenlm: filename=kenlm-0.0.0-cp37-cp37m-linux_x86_64.whl size=2334225 sha256=bcb398d4483db49962722cc50197f7a865406ee73bed892f728ea3df8145536e
  Stored in directory: /tmp/pip-ephem-wheel-cache-ucdvfokf/wheels/3d/aa/02/7b4a2eab5d7a2a9391bd9680dbad6270808a147bc3b7

#SETUP:

In [5]:
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer(lang='ru')

In [6]:
import numpy as np
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch import softmax, sigmoid
from tqdm.auto import tqdm
import argparse
from pymystem3 import Mystem
import sys
from collections import Counter
from functools import partial
import kenlm

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/YandexCup/trained_roberta")

model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/YandexCup/trained_roberta").cuda()

lm = kenlm.Model('/content/drive/MyDrive/YandexCup/lm.binary')



TOXIC_CLASS=-1
TOKENIZATION_TYPE='sentencepiece'


ALLOWED_ALPHABET=list(map(chr, range(ord('а'), ord('я') + 1)))
ALLOWED_ALPHABET.extend(map(chr, range(ord('a'), ord('z') + 1)))
ALLOWED_ALPHABET.extend(list(map(str.upper, ALLOWED_ALPHABET)))
ALLOWED_ALPHABET = set(ALLOWED_ALPHABET)



def logits_to_toxic_probas(logits):
    if logits.shape[-1] > 1:
        activation = lambda x: softmax(x, -1)
    else:
        activation = sigmoid
    return activation(logits)[:, TOXIC_CLASS].cpu().detach().numpy()


def is_word_start(token):
    if TOKENIZATION_TYPE == 'sentencepiece':
        return token.startswith('▁')
    if TOKENIZATION_TYPE == 'bert':
        return not token.startswith('##')
    raise ValueError("Unknown tokenization type")


def normalize(sentence, max_tokens_per_word=20):
    def validate_char(c):
        return c in ALLOWED_ALPHABET
    
    sentence = ''.join(map(lambda c: c if validate_char(c) else ' ', sentence.lower()))
    ids = tokenizer(sentence)['input_ids']
    tokens = tokenizer.convert_ids_to_tokens(ids)[1:-1]
    
    result = []
    num_continuation_tokens = 0
    for token in tokens:
        if not is_word_start(token):
            num_continuation_tokens += 1
            if num_continuation_tokens < max_tokens_per_word:
                result.append(token.lstrip('#▁'))
        else:
            num_continuation_tokens = 0
            result.extend([' ', token.lstrip('▁#')])
    
    return ''.join(result).strip()


def iterate_batches(data, batch_size=40):
    batch = []
    for x in data:
        batch.append(x)
        if len(batch) >= batch_size:
            yield batch
            batch = []
    if len(batch) > 0:
        yield batch


def predict_toxicity(sentences, batch_size=5, threshold=0.5, return_scores=False, verbose=True, device='cuda'):
    results = []
    for batch in iterate_batches(sentences, batch_size):
        normlized = [normalize(sent, max_tokens_per_word=5) for sent in batch]
        tokenized = tokenizer(normlized, return_tensors='pt', padding=True, max_length=512, truncation=True)
    
        logits = model.to(device)(**{key: val.to(device) for key, val in tokenized.items()}).logits
        preds = logits_to_toxic_probas(logits)
        if not return_scores:
            preds = preds >= threshold
        results.extend(preds)
    return results

#EMBEDDINGS:

In [7]:
def get_w2v_indicies(a):
    res = []
    if isinstance(a, str):
        a = a.split()
    for w in a:
        if w in embs_voc:
            res.append((w, embs_voc[w]))
        else:
            
            for lemma in stemmer.lemmatize(w):
                if lemma.isalpha():
                    res.append((w, embs_voc.get(lemma, None)))
    return res


def calc_embs(words):
    words = ' '.join(map(normalize, words))
    inds = get_w2v_indicies(words)
    return [(w, i if i is None else embs_vectors[i]) for w, i in inds]


def calc_single_embedding_dist(word1, word2):
    if (word1 in COMMENTS_VOCAB) and (word2 in COMMENTS_VOCAB[word1]):
        
        return COMMENTS_VOCAB[word1][word2]
    if (word2 in COMMENTS_VOCAB) and (word1 in COMMENTS_VOCAB[word2]):
        
        return COMMENTS_VOCAB[word2][word1]

    a = calc_embs([word1])[0]
    b = calc_embs([word2])[0]



    a_s, a_v = a  #слово, вектор
    b_s, b_v = b  #слово, вектор
    if a_s == b_s: #если слова равны
        if word1 in COMMENTS_VOCAB:
            COMMENTS_VOCAB[word1][word2] = 0.0
        if word2 in COMMENTS_VOCAB:
            COMMENTS_VOCAB[word2][word1] = 0.0
        return 0.0
    if a_v is None or b_v is None: #если одно из векторов нет - дать пизды 
        if word1 in COMMENTS_VOCAB:
            COMMENTS_VOCAB[word1][word2] = 1.0
        if word2 in COMMENTS_VOCAB:
            COMMENTS_VOCAB[word2][word1] = 1.0
        return 1.0
    a = a_v
    b = b_v
    # inexact match is punished by 0.1
    result = 0.1 + 0.9 * (1 - a.dot(b) / np.linalg.norm(a) / np.linalg.norm(b)) / 2

    if word1 in COMMENTS_VOCAB:
        COMMENTS_VOCAB[word1][word2] = result
    if word2 in COMMENTS_VOCAB:
        COMMENTS_VOCAB[word2][word1] = result

    return result


def greedy_match_embs(a, b, max_dist=99999, cache=None, a_ind=0, b_ind=0):
    a_len = len(a) - a_ind #сколько слов мы еще не рассмотрели в а
    b_len = len(b) - b_ind #сколько слов мы еще не рассмотрели в b
    minlen = min(a_len, b_len) #минимальная длина строки
    maxlen = max(a_len, b_len) #минимальная длина строки
    if minlen == 0: 
        return np.minimum(maxlen, max_dist) 
    if maxlen - minlen >= max_dist: 
        return max_dist 
    
    if cache is None:
        cache = {}
    
    cache_key = (a_len, b_len)
    if cache_key in cache:
        return cache[cache_key]
        
    min_dist = max_dist
    first_dist = calc_single_embedding_dist(a[a_ind], b[b_ind])
    if max_dist >= first_dist:
        min_dist = np.minimum(min_dist, first_dist + greedy_match_embs(
            a, b, max_dist, cache, a_ind + 1, b_ind + 1
        ))
    
    if first_dist > 0 and max_dist >= 1:
        min_dist = np.minimum(min_dist, 1 + greedy_match_embs(
            a, b, max_dist, cache, a_ind + 1, b_ind
        ))
        min_dist = np.minimum(min_dist, 1 + greedy_match_embs(
            a, b, max_dist, cache, a_ind, b_ind + 1
        ))
    
    cache[cache_key] = min_dist
    
    return min_dist





def distance_score(original, fixed):
    original = original.split()
    fixed = fixed.split()

    clip_distance = 5  # this clips long computations
    
    numerator = greedy_match_embs(original, fixed, max_dist=clip_distance)

    denominator = 0.6 * np.log(1 + len(original))

    return np.exp(-(numerator / denominator) ** 2)

In [8]:
def compute_lmdiff(original, fixed):
    original_lm_logproba = lm.score(original, bos=True, eos=True)
    fixed_lm_logproba = lm.score(fixed, bos=True, eos=True)
    
    probability_fraction = 10**((fixed_lm_logproba - original_lm_logproba) / 25)
    
    return np.clip(probability_fraction, 0.0, 1.0)
    

def compute_score(original_sentences, fixed_sentences, threshold=0.5, batch_size=5, show_res = False, return_result = False):
    fixed_toxicities = predict_toxicity(fixed_sentences, threshold=threshold, batch_size=batch_size, return_scores=True)
    scores = []
    lmdiffs = []
    emb_dists = []
    for original_sentence, fixed_sentence, fixed_toxicity in tqdm(zip(
        original_sentences, fixed_sentences, fixed_toxicities
    ), miniters=250, disable= True):
        original_sentence = normalize(original_sentence)
        fixed_sentence = normalize(fixed_sentence)
        
        distance = distance_score(original_sentence, fixed_sentence)
        lmdiff = compute_lmdiff(original_sentence, fixed_sentence)
        
        score = (1 - fixed_toxicity) * distance * lmdiff
        
        lmdiffs.append(lmdiff)
        emb_dists.append(distance)
        scores.append(score)
        
    if show_res:
        print('average toxicity:', np.mean(fixed_toxicities), file=sys.stderr)
        print('mean lmdiff:', np.mean(lmdiffs), file=sys.stderr)
        print('mean distance_score:', np.mean(emb_dists), file=sys.stderr)
        print('score', np.mean(scores) * 100, file=sys.stderr)
        
    if return_result:
        return {"scores" : scores, 
                "toxicities" : fixed_toxicities, 
                "lmdiffs" : lmdiffs,
                "distance_scores" : emb_dists}
    else:
        return scores

def new_checker(original_texts, fixed_texts, device='cuda', show_res = False, return_result = False):
    original_texts = list(map(str.strip, original_texts))
    fixed_texts = list(map(str.strip, fixed_texts))
    
    assert len(original_texts) == len(fixed_texts)
    
    with torch.inference_mode(True):
        if return_result:
            return compute_score(original_texts, fixed_texts, show_res = show_res, return_result=return_result)
        return compute_score(original_texts, fixed_texts, show_res = show_res)


In [9]:
embs_file = np.load(DRIVE_PATH + 'embeddings_with_lemmas.npz', allow_pickle=True)
embs_vectors = embs_file['vectors']
embs_voc = embs_file['voc'].item()


embs_voc_normalized = set([normalize(word) for word in tqdm(embs_voc.keys())])
print(len(embs_voc_normalized))

  0%|          | 0/162690 [00:00<?, ?it/s]

148576


In [10]:
texts = []
with open(DRIVE_PATH + 'public_testset.txt', 'rt') as f:
    for line in f:
        texts.append(normalize(line))

comments_voc = set()
for text in texts:
    comments_voc.update(text.split())
print(len(comments_voc))

Token indices sequence length is longer than the specified maximum sequence length for this model (534 > 512). Running this sequence through the model will result in indexing errors


22005


#Обучение супер словаря и toxicity 

In [11]:
EMBEDDING_VOCAB = embs_voc_normalized | comments_voc
print(len(EMBEDDING_VOCAB))

163532


In [12]:
def improve_toxicity(data):

    words = set()
    for text in data:
        words.update(text.split())
    words = sorted(words)

    with torch.inference_mode():
        word_toxicities = predict_toxicity(words, batch_size=100, return_scores=True)

    before = len(toxicity)
    for word, tox in zip(words, word_toxicities):
        toxicity[word] = tox
    print("New words :", len(toxicity) - before)
    print("Cur len :", len(toxicity))

In [13]:
#Уменьшаем EMBEDDING_VOCAB в зависимости от токсичности
toxicity = dict()
improve_toxicity(EMBEDDING_VOCAB)

New words : 155503
Cur len : 155503


In [14]:
NONTOXIC_EMBEDDING_VOCAB = set()

In [15]:
#ТОП НЕТОКСИЧНЫХ ИЗ ЭМБЕДДИНГОВ
NONTOXIC_EMBEDDING_VOCAB .update({word for word, tox in toxicity.items() if tox < 0.0099})
print(len(NONTOXIC_EMBEDDING_VOCAB))

171


In [16]:
#РУЧКАМИ ПОДОБРВАЛИ
SPECIAL_WORDS = ["спасибо", "любовь", "он", 'cardboard', 'замечательную', 'software', 'workbook', "спасибочки", "textbook","я"]
for word in SPECIAL_WORDS:
    NONTOXIC_EMBEDDING_VOCAB.add(word)
print(len(NONTOXIC_EMBEDDING_VOCAB))

176


In [17]:
import random
#ТОП РУССКИХ
top_russian = set()
with open(DRIVE_PATH + 'dicitionary/top_russian.txt', 'rt') as f:
    for line in f:
        w = normalize(line)
        if w and w in embs_voc_normalized:
            top_russian.add(normalize(line))

word_toxicities = predict_toxicity(top_russian, batch_size=100, return_scores=True)
top_russian = set(word for word, tox in zip(top_russian, word_toxicities) if tox < 0.01)
#top_russian = random.sample(top_russian, 500)

NONTOXIC_EMBEDDING_VOCAB.update(top_russian)
print(len(NONTOXIC_EMBEDDING_VOCAB))

178


In [18]:
#ЧАСТЫЕ ИЗ КОММЕНТОВ

d = Counter(' '.join(texts).split())
most_common_words = {word for word, count in d.items() if count > 15}
NONTOXIC_EMBEDDING_VOCAB.update(most_common_words)

print(len(NONTOXIC_EMBEDDING_VOCAB))

606


In [19]:
#СТОП-СЛОВА
import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords


russian_stopwords = []
for word in stopwords.words("russian"):
    w = normalize(word)
    if w and w in embs_voc_normalized:
        russian_stopwords.append(w)


print(len(russian_stopwords))
NONTOXIC_EMBEDDING_VOCAB.update(set(russian_stopwords))
print(len(NONTOXIC_EMBEDDING_VOCAB))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
145
614


In [20]:
COMMENTS_VOCAB = dict()

In [21]:
#Обучаем COMMENTS_VOCAB
for word in NONTOXIC_EMBEDDING_VOCAB | comments_voc:
    if word not in COMMENTS_VOCAB:
        COMMENTS_VOCAB[word] = {} 
print(len(COMMENTS_VOCAB))


22171


In [22]:
improve_toxicity(NONTOXIC_EMBEDDING_VOCAB | comments_voc)

New words : 1
Cur len : 155504


In [ ]:
#for key in tqdm(NONTOXIC_EMBEDDING_VOCAB):
#    for word in comments_voc:
#        if word and key:
#            COMMENTS_VOCAB[key][word] = calc_single_embedding_dist(key, word)

In [ ]:
#Дообучаем COMMENTS_VOCAB
#dsts = []
#for text in tqdm(texts):
#    dsts.append(distance_score(text, text))

#print(sum(dsts)/len(dsts))

In [ ]:
#СОХРАНИТЬ
#import pickle as pkl

#with open(DRIVE_PATH + 'EMBS.pkl', 'wb') as f:
#    pkl.dump(COMMENTS_VOCAB, f)

In [24]:
import pickle as pkl

with open(DRIVE_PATH + 'EMBS.pkl', 'rb') as f:
    COMMENTS_VOCAB = pkl.load(f)

In [ ]:
def get_top_n(word, n):
    candidates = []
    for candidate in NONTOXIC_EMBEDDING_VOCAB:
        if candidate in COMMENTS_VOCAB and word in COMMENTS_VOCAB[candidate]:
            dist = COMMENTS_VOCAB[candidate][word]
            #tox = toxicity[candidate]
            score = (1 - dist) #+ (1 - tox)*1.1
            
            candidates.append([score, candidate])
    candidates.sort(reverse = True)
    #print(candidates[:n])
    return [word for score, word in candidates[:n]]

get_top_n("найти", 30)

#ИЛИ Загрузка этого словаря

In [26]:
def sort_by_toxicity(words):
    toxicities = [toxicity.get(w) if w in toxicity else predict_toxicity([w], return_scores = True) for w in words]
  
    result = [[toxicities[i], i, words[i]] for i in range(len(words))]
    result.sort(reverse = True)
    return result



line = np.random.choice(texts)
pred = predict_toxicity([line])
line_words = normalize(line).split()
print(sort_by_toxicity(line_words))

[[0.7425133, 4, 'курносым'], [0.3129457, 7, 'личиком'], [0.19012807, 5, 'простовато'], [0.1600703, 6, 'миловидным'], [0.08559225, 3, 'е'], [0.0784305, 1, 'только'], [0.07086799, 9, 'смешно'], [0.06700629, 10, 'выглядит'], [0.0640167, 2, 'с'], [0.05111447, 8, 'это'], [0.049239073, 0, 'точно']]


In [27]:
def sort_by_distance(words, word):
    result = [[calc_single_embedding_dist(word, words[i]), i, words[i]] for i in range(len(words))]
    result.sort()
    return result

print(texts[0])
sort_by_distance(texts[0].split(), "все")

он скоро сдохнет и все вернется


[[0.0, 4, 'все'],
 [0.3022717386484146, 0, 'он'],
 [0.32892826944589615, 3, 'и'],
 [0.35516649633646014, 1, 'скоро'],
 [0.39043619036674504, 5, 'вернется'],
 [0.42944292575120924, 2, 'сдохнет']]

In [36]:
def save_comments(comments, name):
    solution = [''] * 2500
    for i in range(len(comments)):
        solution[i] = comments[i]
    with open(DRIVE_PATH + f'{name}.txt', 'wt') as f:
        for text in solution:
            print(text, file=f)

In [29]:
def random_letter(word):
    result_word = list(word)
    rand_letter = np.random.choice(list('абвгдежзийклмнопрстуфхцчшщъыьэюя'))
    rand_idx = np.random.randint(0, len(result_word))
    result_word[rand_idx] = rand_letter
    result_word = ''.join(result_word)
    return result_word

In [30]:
def detox_dumb(i, line):
    words = normalize(line).split()
    sorted_words = sort_by_toxicity(words)#sort_by_distance(words, "спасибо")
    
    score = new_checker([texts[i]], [' '.join(words)])
    for toxic_toxicity, toxic_idx, toxic_word in tqdm(sorted_words):
        

        cur_word = toxic_word
        
        candidates = NONTOXIC_EMBEDDING_VOCAB
        
        for candidate in candidates:
            words[toxic_idx] = candidate
            new_score = new_checker([texts[i]], [' '.join(words)])

            if new_score > score:
                print(score, "->", new_score, candidate)
                score = new_score
                cur_word = candidate
                
            else:
                words[toxic_idx] = cur_word


    cur_scores.append(score[0])
    return ' '.join(words)


In [31]:
def detox_closest(i, line):
    words = normalize(line).split()
    score = new_checker([texts[i]], [' '.join(words)])

    word_to_check = words[:]
    while word_to_check:
        min_dist = 1

        for candidate in NONTOXIC_EMBEDDING_VOCAB:
            
            dist, idx, word = sort_by_distance(word_to_check, candidate)[0]

            if candidate == word:
                continue

            #if toxicity[candidate] > 0.1:# toxicity[word]:
            #    continue

            if dist <= min_dist:
                min_dist = dist
                best_pair = [candidate, word, idx]
        
        
        candidate, word, idx = best_pair
        
        
        new_words = words[:]
        
        new_words[idx] = candidate
        new_score = new_checker([texts[i]], [' '.join(new_words)])
        if new_score > score:
            score = new_score
            words = new_words
            
        

        else:

            word_to_check.remove(word)
    cur_scores.append(score[0])
    return ' '.join(words)

In [ ]:
ind = 654
o = texts[ind]
d = detox_dumb(ind, texts[ind])

new_checker([o], [d], show_res= True)

average toxicity: 0.48775944
mean lmdiff: 1.0
mean distance_score: 0.7995520331989361
score 40.956298032562


[0.40956298032562]

In [ ]:
Original = []
Detoxed = []
for i in tqdm(range(0, len(texts), 50)):
    line = texts[i]
    detoxed = detox_dumb(i, line)

    Original.append(line)
    Detoxed.append(detoxed)


new_checker(Original, Detoxed, show_res = True)

In [97]:
print(i)

334


In [ ]:
#cur_scores = []
#fixed_texts = []
for i in tqdm(range(len(texts))):
    fixed_texts.append(improve_lmdiff(i, texts[i]))
    print(sum(cur_scores)/len(cur_scores), f"{i+1}/2500")


    save_comments(fixed_texts, "submits_pro/every_word_lemma_original")
    

new_checker(texts, fixed_texts, show_res = True)

In [ ]:
save_comments(fixed_texts, "sgit")

#Merges solutions into ultimate one:

In [113]:
ultimate_solution = []

names = os.listdir(path="/content/drive/MyDrive/YandexCup/submits_pro")
if '.ipynb_checkpoints' in names:
    names.remove('.ipynb_checkpoints')


solutions = {name : [] for name in names}
count = {name : 0 for name in names}
#reading submits
for name in names:
    sol = []
    with open(DRIVE_PATH + f'submits_pro/{name}', 'rt') as f:
        for line in f:
            sol.append(normalize(line))

    solutions[name] = sol[:]
    print(name, len(sol))

for i in tqdm(range(len(texts))):
    scores = {name : 0 for name in names}
    for name in names:
        scores[name] = new_checker([texts[i]], [solutions[name][i]])

    best_sol_name = max(scores, key=scores.get)
    count[best_sol_name] += 1
    ultimate_solution.append(solutions[best_sol_name][i])


print(count)
print(new_checker(texts, ultimate_solution, show_res = True))

every_word_lemma.txt 2500
every_word_lemma_2.txt 2500
63_28.txt 2500


  0%|          | 0/2500 [00:00<?, ?it/s]

{'every_word_lemma.txt': 2415, 'every_word_lemma_2.txt': 85, '63_28.txt': 0}
[0.6617002717047347, 0.4600895280633377, 0.6815673165763123, 0.5867507106050701, 0.49972056471916093, 0.4212415034142393, 0.48439955097537774, 0.6423515772572911, 0.5775803791450443, 0.6958473352537378, 0.8118665329906156, 0.7768704111398261, 0.989504829980433, 0.5817849686855852, 0.75453657251864, 0.5710830923677166, 0.4687940247215209, 0.4424448184873531, 0.8140963473304403, 0.5571535844355658, 0.36250081403135276, 0.6669678378726531, 0.09313728207797731, 0.9106083174543708, 0.8945452551014978, 0.7717522562549842, 0.6637871825673999, 0.608366266520608, 0.4001710969370666, 0.8128051862684308, 0.6498369489217634, 0.6479151710867672, 0.3918898204006275, 0.39298208164604076, 0.6467031323680132, 0.8203422134278903, 0.5723236956309949, 0.8593822709887015, 0.4537741319550321, 0.7847193046572881, 0.8864415529795057, 0.6844049769169752, 0.7098272103683679, 0.4602007650388985, 0.4950700686672595, 0.883108260285511, 0.

average toxicity: 0.17656912
mean lmdiff: 0.986273309250579
mean distance_score: 0.7734220076823405
score 63.43501206721165


In [114]:
save_comments(ultimate_solution, "submits_pro/63_43")

In [115]:
63.44 + 47

110.16999999999999

#SUPER BRUTE FORCE:

In [116]:
solution = []

with open(DRIVE_PATH + 'submits_pro/63_43.txt', 'rt') as f:
    for line in f:
        solution.append(normalize(line))

In [108]:
result = new_checker(texts, solution, return_result = True, show_res=True)
print(result)
print(100*np.mean(result["scores"]))

{'scores': [0.6617002717047347, 0.4600895280633377, 0.6815673165763123, 0.5867507106050701, 0.49972056471916093, 0.4212415034142393, 0.48439955097537774, 0.6423515772572911, 0.5775803791450443, 0.6958473352537378, 0.8118665329906156, 0.7768704111398261, 0.989504829980433, 0.5817849686855852, 0.75453657251864, 0.5710830923677166, 0.4687940247215209, 0.4424448184873531, 0.8140963473304403, 0.5571535844355658, 0.36250081403135276, 0.6669678378726531, 0.09313728207797731, 0.9106083174543708, 0.8945452551014978, 0.7717522562549842, 0.6637871825673999, 0.608366266520608, 0.4001710969370666, 0.8128051862684308, 0.6498369489217634, 0.6479151710867672, 0.3918898204006275, 0.39298208164604076, 0.6467031323680132, 0.8203422134278903, 0.5723236956309949, 0.8593822709887015, 0.4537741319550321, 0.7847193046572881, 0.8864415529795057, 0.6844049769169752, 0.7098272103683679, 0.4602007650388985, 0.4950700686672595, 0.883108260285511, 0.8249198696971412, 0.8401973109355396, 0.5685301957728472, 0.070690

average toxicity: 0.18234904
mean lmdiff: 0.9860243808656513
mean distance_score: 0.7773435388370397
score 63.282559159051644


In [ ]:
sorted_by_score = [[score, i, solution[i]] for i, score in enumerate(result["scores"])]
#sorted_by_score = [[toxicity, i, solution[i]] for i, toxicity in enumerate(result["toxicities"])]
sorted_by_score.sort(reverse=False)

np.random.shuffle(sorted_by_score)
skipped = 0
for score, i, comment in tqdm(sorted_by_score):
    print(i, score, comment)
    if len(comment.split()) <= 40000:
        solution[i] = detox_dumb(i, comment)
    else:
        skipped += 1
    
    save_comments(solution, "calcs_1")

In [121]:
def closest_similar(word):
    morphem = morph.parse(word)[0]
    morphems = set()
    for i in morphem.lexeme:
        
        morphems.add(i.word)
    return morphems

print(closest_similar('старческого'))

{'старческих', 'старческие', 'старческом', 'старческою', 'старческого', 'старческая', 'старческую', 'старческий', 'старческое', 'старческим', 'старческому', 'старческой', 'старческими'}


In [139]:
import re
def improve_lmdiff(i, line):
    original = normalize(texts[i]).split()

    words = normalize(line).split()
    score = new_checker([texts[i]], [' '.join(words)])
    
    for j in range(len(original)):

        cur_word = words[j]

        if original[j] != cur_word or not re.sub(r'[^а-я]', '', original[j]):
            
            

            candidates = set()
            wrds = ['они', 'дизайн', 'замечательный', "он", "любовь", original[j]]
            for w in wrds:
                candidates.update(closest_similar(w))
            
        else:

            candidates = closest_similar(words[j])
        
        for candidate in candidates:
            words[j] = candidate
            
            new_score = new_checker([texts[i]], [' '.join(words)])

            if new_score > score:
                print(score, "->", new_score) 
                print(cur_word, "->", candidate)
                score = new_score
                cur_word = candidate
                
            else:
                words[j] = cur_word

    
    cur_scores.append(score[0])
    return ' '.join(words)



In [49]:
count = 0
for i in range(len(texts)):
    if len(texts[i].split()) != len(solution[i].split()):
        
        count += 1
print(count)

14


#Параллельный брутфорс

In [ ]:
for i in tqdm(range(1250)):
    comment = solution[i]
    print(i, comment)
    if len(texts[i].split()) == len(solution[i].split()):
        solution[i] = improve_lmdiff(i, comment)

    
        save_comments(solution, "submits_pro/every_word_lemma")

  0%|          | 0/1250 [00:00<?, ?it/s]

0 он скоро ну и все вернется
1 а т у т а на заголо маринования стать и р е а к ц и я тако в а рви т е вс ю сис тему и бан ки и по ли ти ку т ре б у й и т е же род н ы е наличны е н а карман вывод и т е весь бе с и нал же а по ли ти ка та ко ва мас ки це ль од на мис что жи ть наш р о д п р о с н и с ь род м о cf же
2 понравился алису нет никакого желания не по божески да но замечателен е надо чтоб ей так жилось как простым замечательно одиночкам
[0.6815673113135026] -> [0.7159674312020682]
понравился -> пожалейте
[0.7159674312020682] -> [0.7216395821861444]
замечательно -> мамку
[0.7216395821861444] -> [0.728385633812562]
мамку -> мамкой
[0.728385633812562] -> [0.7306843018467267]
мамкой -> мамках
[0.7306843018467267] -> [0.7562878123547838]
мамках -> мамок
3 наука в стране никогда не нужна
4 спасибочки выпей а то дизайн от злости
[0.4997205858185629] -> [0.5153732819061472]
дизайн -> замечательно
5 а меня не смутил яд на любовь возможно он попал на guestbook лешеньки прямиком с губ сп

In [78]:
save_comments(solution, "submits_pro/full_lemma")